# 목적
* 크롤링 결과로 네트워크분석
* [참고](https://foreverhappiness.me/38)


# 선행설치

In [1]:
# import
import seaborn as sns
import matplotlib.pyplot as plt

import networkx as nx
import operator
import random

from tqdm import tqdm_notebook  # 진행과정 시각화
from datetime import timedelta  # 시간날짜

import datetime
import pandas as pd
import numpy as np
import os
import kss
import re
import matplotlib.font_manager as fm
import gc

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings(action='ignore')

#한글깨짐방지
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important; }</style>'))

In [2]:
def word_couple(df, year,stopword, custom_sw,keyword,verbose = False):
    ''' 데이터프레임을 전달하면 단어쌍을 계산
    명사에 한정해서 진행하며, 연도별로 나눌수 있게끔 진행
    
    Parameter
    ---------
    df(DataFrame) : 크롤링된 데이터프레임
    year(int) : 분석할 연도
    stopword(list) : 불용어
    custom_sw(dictionary) : 단어별 개별 불용어
    keyword(string) : 불용어처리를 위한 키워드
    verbose(bool): 진행과정 확인
    
    Return
    ------
    word_couple_df : DataFrame
        word1    word2    freq
    0   서핑     양양군   1187
        
    
    '''
    import matplotlib.font_manager as fm
    
    #stopword
    stopword = stopword +[keyword]
    try :
        stopword = stopword +custom_sw[keyword]
        stopword = list(set(stopword))
    except:
        pass
    
    
    df = df.fillna("")    
    df['Noun'] = df['Noun'].apply(lambda x : re.sub("[\[\]' ]","",x).split(','))
    
    year_noun = df.loc[df.year == year,'Noun']
    year_noun = year_noun.apply(lambda sentence : [word for word in sentence if word not in stopword])
    corpus =list(map(lambda words : " ".join(words),year_noun))
    
    #DTM
    vector = CountVectorizer()
    vector.fit(corpus) #코퍼스로부터 각 단어의 빈도 수를 기록한다.
    values = vector.transform(corpus).toarray()
    cols = vector.get_feature_names() # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.
    DTM_df = pd.DataFrame(values,columns= cols)
    
    # 단어의수가 70~80개가 되는 적정범위 산출
    
    imsi = DTM_df.sum()
    count = 0
    fin = 0
    start = 0
    end = imsi.max()
    min_v = 70
    max_v = 80
    while (fin>max_v)|(fin<min_v):
        y = random.randint(start, end)
        fin = imsi[imsi>y].shape[0]

        if fin <= min_v:
            end = y
        elif fin >= max_v:
            start = y
            
        if count> 15:
            break
    count_min = y

    select_cols = imsi[imsi>count_min].index.values
    DTM_df = DTM_df[select_cols]
    cols = DTM_df.columns.values
    word_length = len(cols)
    
    #단어쌍 계산
    count_dict = {}
    for i in range(len(DTM_df)): #각 블로그글 
        sample = DTM_df.T.iloc[:,i]
        sample = sample[sample>0]
        len_max = len(sample)
        for j in range(len_max):
            for z in range(j+1,len_max):
                count_dict[sample.index[j], sample.index[z]] = count_dict.get((sample.index[j], sample.index[z]), 0) + max(sample[sample.index[j]], sample[sample.index[z]])

    count_list = []
    for words in count_dict:
        count_list.append([words[0],words[1],count_dict[words]])
    word_couple_df = pd.DataFrame(count_list, columns = ['word1','word2','freq'])
    word_couple_df = word_couple_df.sort_values(by='freq',ascending= False)
    word_couple_df.reset_index(drop =True,inplace = True)
    
    if verbose == True:
        print(keyword, year)
        print("연도별 대상 row수:",year_noun.shape[0])
        print("count_min:", count_min)
        print("단어 길이:", word_length)
        print("word_couple_df 길이:",word_couple_df.shape[0])
        print("------------------")
    
    return word_couple_df

In [3]:
def networkx_graph(df, keyword, year, font_size = 18, k = 7, save=True):
    '''데이터프레임을 전달하면 네트워크분석 진행
    word_couple함수를 통해 나온 결과를 전달하면, 네트워크분석결과를 전달
    
    Parameter
    ---------
    df(DataFrame) : word_couple함수의 결과인 데이터프레임 전송
    keyword(string) : 파일저장시 사용할 keyword
    year(int) : 파일저장시 사용할 year
    min_cuple(int) : 최소 frq의 크기 (단어의 개수를 결정)
    font_size(int) : 출력되는 폰트의 크기
    k(int) : 노드간의 거리 조절(클수록 거리가 멀어짐)
    save(bool) : 저장 결정여부 
    
    Return
    ------
    G : network 출력 결과 
    '''
    import matplotlib.font_manager as fm
    
    # 적정 min_couple 70~80산출
    count = 0
    fin = 0
    start = 0
    end = df.freq.max()
    min_v = 70
    max_v = 80

    while (fin >max_v)|(fin<min_v):
        y = random.randint(start, end)
        fin = df[df.freq>y].shape[0]
        
        count +=1
        if fin <= min_v:
            end = y
        elif fin >= max_v:
            start = y
            
        if count >15:
            break
    min_couple = y
    
    #network
    G_centrality = nx.Graph() #중심성척도계산을 위한 그래프
    for ind in range((len(np.where(df['freq'] >=min_couple)[0]))):
        G_centrality.add_edge(df['word1'][ind],df['word2'][ind], weight = int(df['freq'][ind])) #word1,word2간의 빈도수를 갖는 edge생성 
    dgr = nx.degree_centrality(G_centrality) #연결중심성(직접적으로 연결된 노드의수 )
   
    #중심성이 큰순으로 정렬 
    sorted_dgr = sorted(dgr.items(),key=operator.itemgetter(1), reverse = True)
  
    #네트워크 그릴 그래프 선언
    G = nx.Graph()
    
    #페이지 랭크에 따라 두 노드사이 연관성 결정
    for i in range(len(sorted_dgr)):
        G.add_node(sorted_dgr[i][0], nodesize = sorted_dgr[i][1])
    for ind in range((len(np.where(df['freq']>=min_couple)[0]))):
        G.add_weighted_edges_from([(df['word1'][ind],df['word2'][ind],int(df['freq'][ind]))])
        
    #노드크기
    sizes = [G.nodes[node]['nodesize']*700 for node in G]
    
    #컬러, 라벨다는 것 등 
    options = {
        'edge_color' : '#FFDEA2',
        'node_color' : '#F09B7B',
        'width':1,
        'with_labels':True,
        'font_weight':'regular'
    }
    #한글 설정
    fm._rebuild()
    font_fname = "c:/Windows/Fonts/malgun.ttf"
    fontprop = fm.FontProperties(fname=font_fname, size = 20).get_name()
    
    #그래프 생성
    plt.figure(figsize= (20,10))
    plt.title(f'{year}',fontsize = 20) 
    nx.draw(G,node_size=sizes, pos = nx.spring_layout(G, k=5.5, iterations= 100), **options, font_family=fontprop, font_size=font_size)
    ax = plt.gca()
    ax.collections[0].set_edgecolor("#555555")
    
    if save == True:
        os.makedirs(f"./output/네트워크분석/{keyword}",exist_ok=True)
        plt.savefig(f"./output/네트워크분석/{keyword}/{keyword}_{year}_networkx.png",bbox_inches='tight')
    plt.close()
    gc.collect()

In [4]:
#stopword
sw = list(pd.read_excel("stopword(cp949).xlsx",encoding = 'cp949')['불용어']) #불용어 불러오기

#개별 불용어 
with open('custom_sw.json') as load_file:
    custom_sw = json.load(load_file)

In [5]:
path = './output/token_통합/'
file_list = os.listdir(path)
for file in tqdm_notebook(file_list):
    df = pd.read_csv(path+file)
    keyword = file.split('_')[0]
    for year in range(2011, 2020,2): #간격조정 가능
        word_couple_df = word_couple(df,year,sw, custom_sw,keyword)
        G = networkx_graph(word_couple_df,keyword, year,font_size = 18, k= 7,save = True)
    gc.collect()

In [6]:
def make_gif(path):
    '''
    이미지 여러장이 들어있는 폴더를 input하면 gif를 만들어냄
    jpg, png파일만 허용
    '''
    from PIL import Image
    import os
    import imageio
    
    file_list = os.listdir(path)
    
    #select png
    png_ls =[]
    for file in file_list:
        try :
            if file.split('.')[1] in (['png','jpg']):
                png_ls.append(file)
        except : 
            pass
    
    #naming
    keyword = png_ls[0].split('_')[0]
    start = png_ls[0].split('_')[1]
    end = png_ls[-1].split('_')[1]
                                      
    images = [np.array(Image.open(path+file)) for file in png_ls]
    imageio.mimsave(f'./output/네트워크분석/{keyword}_{start}~{end}.gif', images, fps=0.5)

In [7]:
path = './output/네트워크분석'

for folder in os.listdir(path):
    if os.path.splitext(folder)[1] == "":
        make_gif(f'{path}/{folder}/')